<a href="https://colab.research.google.com/github/iraidaantropova/data-processing-method/blob/main/lm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import scrapy
from scrapy.http import HtmlResponse
from lm_parser.items import LmParserItem
from scrapy.loader import ItemLoader

class LmSpider(scrapy.Spider):
    name = 'lm'
    allowed_domains = ['leroymerlin.ru']

    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.start_urls = [f'https://leroymerlin.ru/search/?q={kwargs.get("search")}']

    def parse(self, response: HtmlResponse):
        links = response.xpath("//a[@data-qa='product-name']")
        for link in links:
            yield response.follow(link, callback=self.parse_lm)

    def parse_lm(self, response: HtmlResponse):
        loader = ItemLoader(item=LmParserItem(), response=response)
        loader.add_xpath('name', "//h1/text()")
        loader.add_xpath('price', "//div//span/text()")
        loader.add_xpath('photos', "//picture/img/@src | //picture/img/@data-src")
        loader.add_value('url', response.url)
        yield loader.load_item()

        #name = response.xpath("//h1/text()").get()
        #price = response.xpath("//div//span/text()").getall()
        #photos = response.xpath("//picture/img/@src | //picture/img/@data-src").getall()
        #url = response.url
        #yield LmParserItem(name=name, price=price, photos=photos, url=url)


